In [1]:
import json
def json_data(data):
    json_data = json.loads(data.to_json())
    json_data = json.dumps(json_data)
    return json_data

In [2]:
import numpy as np
import colorcet as cc
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.models import Legend
from bokeh.models import ColumnDataSource, LinearColorMapper
from bokeh.io import curdoc, output_notebook, show
from bokeh.models import LinearColorMapper
from bokeh.models import BasicTickFormatter, LogTicker,ColorBar
from bokeh.models import LogColorMapper

### Preparing the data

In [3]:
%run data_prep.ipynb

In [4]:
dead = dead_per_million_df
countries = dead['country']
dead = dead.drop(['country','country_code','geometry','population'], axis=1)
latest = dead[dead.columns[-1]]

In [5]:
dead.fillna(0)
dead.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.177562,0.177562,0.177562,0.177562,0.284099,0.284099,0.284099,0.284099,0.284099,0.284099
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,69.351187,72.481435,76.637110,80.468965,85.110368,89.293027,95.418599,99.385378,102.380702,107.993561
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,164.545628,167.992347,172.210445,178.626363,186.621771,192.832599,198.792423,203.157451,207.176582,210.972259


In [6]:
indexed_data = dead.loc()

indexed_data[0]

1/22/20    0.0
1/23/20    0.0
1/24/20    0.0
1/25/20    0.0
1/26/20    0.0
          ... 
4/30/20    0.0
5/1/20     0.0
5/2/20     0.0
5/3/20     0.0
5/4/20     0.0
Name: 0, Length: 104, dtype: float64

In [7]:
counter = countermeasures_df
counter = counter.drop(counter.columns[11:], axis=1)
counter = counter.drop('country_code', axis=1)
current_deaths = countermeasures_df['ConfirmedDeaths'].fillna(0)
counter = counter.join(current_deaths)
counter = counter.join(countermeasures_df['StringencyIndexForDisplay'].fillna(0))

counter

,CountryName,Date,C1_School_closing,C2_Workplace_closing,C3_Cancel_public_events,C4_Restrictions_on_gatherings,C5_Close_public_transport,C6_Stay_at_home_requirements,C7_Restrictions_on_internal_movement,C8_International_travel_controls,ConfirmedDeaths,StringencyIndexForDisplay
0,Aruba,1/1/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,1/2/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,1/3/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,1/4/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,1/5/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19318,Kosovo,5/2/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,83.6
19319,Kosovo,5/3/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,83.6
19320,Kosovo,5/4/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,83.6
19321,Kosovo,5/5/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,83.6


### Helper functions

In [8]:
# the plots legend cannot display all counties, so this is a smaller list that we are interested in
interest = ['United States of America','United Kingdom','Japan','China','Germany','Russia',
            'India','Italy','Australia','Brazil','France','Spain','Denmark','Sweden','Norway']

In [9]:
# Some contries are not named the same in the two dataframes
def translator(strr):
    if strr=='United States':
        return'United States of America'
    
    if strr=='United States of America':
        return'United States'
        
    if strr=='Serbia':
        strr='Republic of Serbia'
        
    if strr=='Democratic Republic of Congo':
        strr='Democratic Republic of the Congo'
    
    if strr=='Dominica':
        strr='Dominican Republic'
        
    if strr=='Kyrgyz Republic':
        strr='Kyrgyzstan'
        
    if strr=='Tanzania':
        strr='United Republic of Tanzania'
        
    if strr=='Eswatini':
        strr='eSwatini'
    return strr

# newest_per_million skal bruges istedet her.

In [10]:
# finds the latest dead pr. million for a given country
def late(strr):
    
    strr=translator(strr)
    
    for i in range(len(countries)):
        if strr==countries[i]:
            return latest[i]/dead_per_million_df['population'][i]*1000000      
    return 

### Process the data

The aim is to find the first day of every countermeasure for every country, but we alto locate the fist day of death for every country. This is used for the scatter plot.

This also sets timestamps for whenever 'StringencyIndexForDisplay' is changed This will be usefull for the graph.

In [11]:
# Used to build a list for when the first countermeasures and deaths occured
arr = np.zeros((len(countries),10))
arr_c = []
days=-1
country = counter['CountryName'][0]
c1,c2,c3,c4,c5,c6,c7,c8,de=[0,0,0,0,0,0,0,0,0]
arr_country=0

# lists used to build index and values for when theres a difference in 'StringencyIndexForDisplay'
arr_str_indx = []
arr_str_iday = []
arr_n = []
arr_d = []
pre = counter['StringencyIndexForDisplay'][0]

for i in range(len(counter['Date'])):
    if i==len(counter['Date']):
        if not de == 0:
            arr[arr_country]=[c1-de,c2-de,c3-de,c4-de,c5-de,c6-de,c7-de,c8-de,de,late(country)]
        arr_c.append(country)
        arr_str_indx.append(arr_n)
        
        break 
    if not country == counter['CountryName'][i]:
        days=-1
        if not de == 0:
            arr[arr_country]=[c1-de,c2-de,c3-de,c4-de,c5-de,c6-de,c7-de,c8-de,de,late(country)]
        arr_c.append(country)
        arr_str_indx.append(arr_n)
        arr_str_iday.append(arr_d)
        arr_n = []
        arr_d = []
        country = counter['CountryName'][i]
        c1,c2,c3,c4,c5,c6,c7,c8,de=[0,0,0,0,0,0,0,0,0]
        arr_country+=1
        pre = counter['StringencyIndexForDisplay'][i]
        
    days+=1
    if not counter['C1_School_closing'][i]==0 and c1==0:
        c1=days
    
    if not counter['C2_Workplace_closing'][i]==0 and c2==0:
        c2=days
        
    if not counter['C3_Cancel_public_events'][i]==0 and c3==0:
        c3=days
        
    if not counter['C4_Restrictions_on_gatherings'][i]==0 and c4==0:
        c4=days
        
    if not counter['C5_Close_public_transport'][i]==0 and c5==0:
        c5=days
    
    if not counter['C6_Stay_at_home_requirements'][i]==0 and c6==0:
        c6=days
        
    if not counter['C7_Restrictions_on_internal_movement'][i]==0 and c7==0:
        c7=days
        
    if not counter['C8_International_travel_controls'][i]==0 and c8==0:
        c8=days
        
    if not counter['ConfirmedDeaths'][i]==0 and de==0:
        de=days
        
    
    if not pre == counter['StringencyIndexForDisplay'][i]:
        pre = counter['StringencyIndexForDisplay'][i]
        if not days-22 < 0:
            arr_d.append(days-22)
            arr_n.append(pre)


We put the collected data into a dataframe.

In [12]:
when_df=pd.DataFrame(arr, columns=['C1_School_closing','C2_Workplace_closing','C3_Cancel_public_events',
                                     'C4_Restrictions_on_gatherings','C5_Close_public_transport',
                                     'C6_Stay_at_home_requirements','C7_Restrictions_on_internal_movement',
                                     'C8_International_travel_controls','ConfirmedDeaths','Total_deaths_pr_million']) 
temp_df=pd.DataFrame(arr_c, columns=['Country'])
when_df=temp_df.join(when_df)

when_df[:]

,Country,C1_School_closing,C2_Workplace_closing,C3_Cancel_public_events,C4_Restrictions_on_gatherings,C5_Close_public_transport,C6_Stay_at_home_requirements,C7_Restrictions_on_internal_movement,C8_International_travel_controls,ConfirmedDeaths,Total_deaths_pr_million
0,Aruba,-31.0,-26.0,-18.0,-18.0,16.0,-26.0,-18.0,-32.0,106.0,NaN
1,Afghanistan,-10.0,0.0,-23.0,12.0,4.0,1.0,1.0,-22.0,83.0,0.065133
2,Angola,-6.0,-6.0,-3.0,-6.0,-3.0,-3.0,-3.0,-53.0,89.0,0.002107
3,Albania,-3.0,-3.0,-3.0,-3.0,1.0,1.0,1.0,-16.0,71.0,3.773074
4,Andorra,-11.0,-13.0,-14.0,-86.0,-2.0,-14.0,41.0,12.0,86.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
145,Vietnam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
146,South Africa,-9.0,-1.0,-11.0,-11.0,-1.0,-1.0,-12.0,-64.0,86.0,0.041336
147,Zambia,-14.0,29.0,12.0,-8.0,29.0,6.0,-5.0,-42.0,93.0,0.009964
148,Zimbabwe,0.0,6.0,-7.0,-7.0,6.0,-1.0,-1.0,-56.0,83.0,0.019186


### The Graph

In [13]:
#Define the graph colors
graph_pal = [cc.rainbow[i] for i in range(len(countries))]
#Define a palette of grays for the points on the graph
gray_pal = [cc.gray[i*2] for i in range(101)]
#Reverse color order 
gray_pal = gray_pal[::-1]

In [14]:
# Converts the StringencyIndexForDisplay data into gray colors
arr_gray = []
for i in range(len(arr_str_indx)):
    gr = []
    for j in range(len(arr_str_indx[i])):
        gr.append(gray_pal[int(arr_str_indx[i][j])])
    arr_gray.append(gr)

In [15]:
# graph with points on
p1 = figure( title="Death pr. mil. over time", plot_width=800, plot_height=500)
p1.xaxis.axis_label = 'Days after Jan 22th'

legend_it = []

for i in range(len(countries)):
    if countries[i] in interest:
        if not when_df.index[when_df['Country']==translator(countries[i])].tolist() == []:
            c = p1.line(range(len(indexed_data[i])), indexed_data[i], line_width=2, color=graph_pal[i], 
                        alpha=0.8, muted_alpha=0.01, muted=True)
            d = []
            x = when_df.index[when_df['Country']==translator(countries[i])].tolist()[0]

            d = p1.circle(arr_str_iday[x], indexed_data[i][arr_str_iday[x]], size=10, 
                      fill_color = arr_gray[x], 
                      line_color=graph_pal[i], line_width=2, muted_alpha=0, muted=True)
            #print(str(j) + " " + str(len(arr_str_indx[i])))
            legend_it.append((countries[i],[c,d]))
        
    
legend = Legend(items=legend_it)
legend.click_policy="mute"
p1.add_layout(legend, 'right')
p1.legend.label_text_font_size = '8pt'

output_notebook()
output_file("./bokeh/graph.html")
show(p1)

Loading BokehJS ...

### The scatter plots

In [16]:
# filter out countries that have yet to have a death
filter_df = when_df[when_df.ConfirmedDeaths != 0]

In [17]:
from bokeh.models import HoverTool
#Define hover tool showing the country name and the death count
hover = HoverTool(tooltips = [ ('Country','@country'),('Deaths', '@ConfirmedDeaths')])

In [18]:
#Define the palette for the scatterplot
scatter_pal = cc.fire[:-25]
#Reverse color order 
scatter_pal = scatter_pal[::-1]

#Define logarithmic color mapper
color_mapper_scatter = LogColorMapper(palette = scatter_pal, low = 0.0001, 
                                      high = max(when_df['Total_deaths_pr_million'].fillna(0)),
                                      low_color = '#fff0ba', nan_color = '#d9d9d9')

In [19]:

#Define colorbar using logarithmic ticks
color_bar = ColorBar(color_mapper=color_mapper_scatter, ticker=LogTicker(),
                     label_standoff=2, border_line_color=None, location=(0,0), 
                     major_label_text_align='left', 
                     formatter = BasicTickFormatter(use_scientific = False,precision=6))


In [20]:
w = 450
h = 270
TOOLS='pan,wheel_zoom,box_zoom,reset'

pp = []

for i in range(1,9):
    n = when_df.columns[i]
    pa = figure( title=n, tools=TOOLS, plot_width=w, plot_height=h)
    pa.add_tools(HoverTool(tooltips=[("Country", "@Country"), ("Current deaths pr million", "@Total_deaths_pr_million")]))
    pa.xaxis.axis_label = 'Days before first confirmed death'
    pa.yaxis.axis_label = 'Relative first day of countermeasure'
    pa.x_range.start = 0
    pa.circle('ConfirmedDeaths', n, size=12, source = filter_df, 
         fill_color = {'field' : 'Total_deaths_pr_million', 'transform' : color_mapper_scatter}, 
          line_color="black", fill_alpha=0.6, line_alpha=0.6)
    if i%2==1:
        pa.add_layout(color_bar, 'left')
    else:
        pa.add_layout(color_bar, 'right')
        
    pp.append(pa)


In [21]:
from bokeh.layouts import column, row

layout = row(
    column(pp[0], pp[2], pp[4], pp[6]),
    column(pp[1], pp[3], pp[5], pp[7])
)

output_notebook()
output_file("./bokeh/scatter.html")
show(layout)

Loading BokehJS ...